This notebook contains the code for the robustness analysis of the paper.

In [ ]:
import tvbsim.printer as printer

In [ ]:
seeds = list(range(50))
n_minimal_seeds = 15
pci_seeds = list(range(50))
n_minimal_seeds_pci = 15
run_sim = 5000.0
cut_transient = 4000.0
run_sim_pci = 8000.0
cut_transient_pci = 4000.0

E_K_e = general_parameters.parameter_model['E_K_e']
E_K_i = general_parameters.parameter_model['E_K_e']
E_Na_e = general_parameters.parameter_model['E_Na_e']
E_Na_i = general_parameters.parameter_model['E_Na_i']
g_L = E_Na_e = general_parameters.parameter_model['g_L']

E_L_e_start = general_parameters.parameter_model['E_L_e']
E_L_i_start = general_parameters.parameter_model['E_L_i']

path = os.getcwd()
connectome = 'DK68' # 'Lausanne463'
if connectome == 'DK68':
    general_parameters.parameter_connection_between_region['path'] = f'{path}/data/connectivity/DK68'
    general_parameters.parameter_connection_between_region['conn_name'] = 'connectivity_68_QL20120814.zip'
    receptors = np.loadtxt('./data/receptors/DK68/5HT2a_reordered.txt')
    stimregion = 5
    n_regions = 68
elif connectome == 'Lausanne463':
    general_parameters.parameter_connection_between_region['path'] = f'{path}/data/connectivity/Lausanne463'
    general_parameters.parameter_connection_between_region['conn_name'] = 'connectivity_cortical_Lausanne463.zip'
    receptors = np.loadtxt('./data/receptors/Lausanne463/5HT2A_Lausanne463_cortical.txt')
    stimregion = 252
    n_regions = 412
else:
    raise Exception('variable connectome should be either DK68 or Lausanne463')

stimval = 0.5e-3 # IN KILO HERTZ
t_analysis = 300
n_trials = 5
np.random.seed(0)
stimtime_allseeds = np.random.randint(cut_transient_pci+t_analysis,
                                      run_sim_pci-t_analysis,
                                      (len(pci_seeds,)))
stimtime_allseeds = [int(i) for i in stimtime_allseeds]
param_change_with_seed = [{'stimtime':stimtime} for stimtime in stimtime_allseeds]

stop_conditions = [(1,0,-np.inf,150*1e-3,2500)]

# Define the range of parameters that will be tested
# step = 0.1
# E_L_e_ends = [np.round(E_L_e_start+step+i, decimals=1) for i in np.arange(0,1,step)]
# E_L_i_ends = [np.round(E_L_i_start+step+i, decimals=1) for i in np.arange(0,1,step)]
step = 0.4
E_L_e_ends = [np.round(E_L_e_start-2+step+i, decimals=1) for i in np.arange(0,5,step)]
E_L_i_ends = [np.round(E_L_i_start-2+step+i, decimals=1) for i in np.arange(0,5,step)]
simconfigs_setups = []
for i,E_L_i_end in enumerate(E_L_i_ends):
    for j,E_L_e_end in enumerate(E_L_e_ends): 
        g_K_e_nopsi,g_Na_e_nopsi = conversion(E_Na=E_Na_e, E_K=E_K_e, E_L=E_L_e_start, g_L=g_L)
        g_K_e_psi,g_Na_e_psi = conversion(E_Na=E_Na_e, E_K=E_K_e, E_L=E_L_e_end, g_Na=g_Na_e_nopsi)
        
        g_K_i_nopsi,g_Na_i_nopsi = conversion(E_Na=E_Na_i, E_K=E_K_i, E_L=E_L_i_start, g_L=g_L)
        g_K_i_psi,g_Na_i_psi = conversion(E_Na=E_Na_i, E_K=E_K_i, E_L=E_L_i_end, g_Na=g_Na_i_nopsi)

        params_nopsi = {'g_K_e':g_K_e_nopsi, 'g_Na_e':g_Na_e_nopsi, 'g_K_i':g_K_i_nopsi, 'g_Na_i':g_Na_i_nopsi}
        params_psi = {'g_K_e':ListParameter(
                        'g_K_e', 
                        get_g_K_values(g_K_e_nopsi, g_K_e_psi, receptors=receptors),
                        f'prop_{E_L_e_start}_{E_L_e_end}'),
                      'g_Na_e':g_Na_e_psi, 
                      'g_K_i':ListParameter(
                        'g_K_e', 
                        get_g_K_values(g_K_i_nopsi, g_K_i_psi, receptors=receptors),
                        f'prop_{E_L_i_start}_{E_L_i_end}'),
                      'g_Na_i':g_Na_i_psi}

        simconfigs = {
            'E_L_e_end':E_L_e_end,
            'E_L_i_end':E_L_i_end,
            'E_L_e_end_idx':j,
            'E_L_i_end_idx':i,
            'ai_nopsi':SimConfig(
                general_parameters=general_parameters, 
                run_sim=run_sim, 
                cut_transient=cut_transient, 
                parameters=params_nopsi, 
                stop_conditions=stop_conditions,
                params_to_report=['g_K_e','g_K_i','g_Na_e','g_Na_i']),
            'ai_psi':SimConfig(
                general_parameters=general_parameters, 
                run_sim=run_sim, 
                cut_transient=cut_transient, 
                parameters=params_psi, 
                stop_conditions=stop_conditions,
                params_to_report=['g_K_e','g_K_i','g_Na_e','g_Na_i'])}

        # PCI
        for config_name,params in [('pci_nopsi', params_nopsi),('pci_psi', params_psi)]:
            stim_params = {'stimval':stimval, 'stimregion':[stimregion], 'stimdur':50}
            params_PCI = {**params, **stim_params}
            simconfigs[config_name] = SimConfig(
                general_parameters=general_parameters,
                run_sim=run_sim_pci,
                cut_transient=cut_transient_pci,
                parameters=params_PCI,
                stop_conditions=stop_conditions,
                params_to_report=['g_K_e','g_K_i','g_Na_e','g_Na_i','stimval','stimregion','stimdur'])
        simconfigs_setups.append(simconfigs)

subfolder = f'{connectome}_Grid'
simu_path = f'./result/{subfolder}'

In [ ]:
# Limits amounts of prints
printer.PRINT_LEVEL = 3

In [ ]:
%matplotlib agg

# First: LZc
# If average(LZc nopsi) > average(LZc_psi) : stop
# Compute PSD for all seeds and plot
# Run PCI simulations

from einops import rearrange

boxplot_params = {'showmeans': False,}
colors = ['#444444', '#4A90E2']
x = ['Baseline','K+ conductance reduction']
vars_int = ['E', 'I']
monitor = 'Raw'
imgs_path_root = f'./img/{subfolder}'

entropy_names = ['LZc','LZc_single']
entropy_deltas = np.zeros((len(entropy_names),len(E_L_e_ends), len(E_L_i_ends)))
entropy_pvalues = np.zeros((len(entropy_names),len(E_L_e_ends), len(E_L_i_ends)))

pci_deltas = np.zeros((len(E_L_e_ends), len(E_L_i_ends)))
pci_pvalues = np.zeros((len(E_L_e_ends), len(E_L_i_ends)))

ai_explosions_path = os.path.join(simu_path, 'ai_explosions.txt')
try:
    ai_explosions = np.load(ai_explosions_path)
except:
    ai_explosions = np.zeros((len(E_L_e_ends), len(E_L_i_ends)), dtype=int)
    
pci_explosions_path = os.path.join(simu_path, 'pci_explosions.txt')
try:
    pci_explosions = np.load(pci_explosions_path)
except:
    pci_explosions = np.zeros((len(E_L_e_ends), len(E_L_i_ends)), dtype=int)

seeds_no_exp_path = os.path.join(simu_path, 'seeds_no_exp.npy')
try:
    seeds_no_exp = np.load(seeds_no_exp_path)
except:
    seeds_no_exp = -np.ones((len(E_L_e_ends), len(E_L_i_ends), n_minimal_seeds), dtype=int)

frequency_differences = np.zeros((len(E_L_e_ends),len(E_L_i_ends),len(BAND_NAMES)))
frequency_pvalues = np.zeros((len(E_L_e_ends),len(E_L_i_ends),len(BAND_NAMES)))

mean_differences = np.zeros((len(E_L_e_ends),len(E_L_i_ends)))
mean_pvalues = np.zeros((len(E_L_e_ends),len(E_L_i_ends)))

std_differences = np.zeros((len(E_L_e_ends),len(E_L_i_ends)))
std_pvalues = np.zeros((len(E_L_e_ends),len(E_L_i_ends)))

for simconfigs_setup in simconfigs_setups:

    E_L_e_end = simconfigs_setup['E_L_e_end']
    E_L_i_end = simconfigs_setup['E_L_i_end']
    e_idx = simconfigs_setup['E_L_e_end_idx']
    i_idx = simconfigs_setup['E_L_i_end_idx']
    print('Computing for params', E_L_e_end, E_L_i_end)
    
    imgs_path = imgs_path_root + f'/{E_L_e_end}_{E_L_i_end}'
    
    simconfigs = simconfigs_setup['ai_nopsi'], simconfigs_setup['ai_psi']
    
    if ai_explosions[e_idx,i_idx] == 1:
        print('This config caused an explosions before.')
        continue
    if os.path.exists(os.path.join(imgs_path, 'config_done.npy')):
        continue
        
    print('Running simulations for LZc...')
    seeds_completed = []
    for simconfig in simconfigs:
        seeds_completed_simconfig = run_n_complete_parallelized(
            simconfig, n_minimal_seeds=n_minimal_seeds, max_seed=len(seeds), save_path_root=simu_path, n_tasks_concurrent=5)
        seeds_completed.append(seeds_completed_simconfig)
    print('Seeds completed:', seeds_completed)

    ai_explosions[e_idx,i_idx] = not bool(seeds_completed[1])
    np.save(ai_explosions_path, ai_explosions)
    if ai_explosions[e_idx,i_idx] == 1:
        print('Not enough seeds completed, stopping this config')
        continue  
    
    seeds_no_exp[e_idx,i_idx] = seeds_completed[1]
    np.save(seeds_no_exp_path, seeds_no_exp)

    # Compute mean & std of firing rates across time of all channels for all seeds
    means = np.zeros((len(simconfigs), n_minimal_seeds*n_regions))
    stds = np.zeros((len(simconfigs), n_minimal_seeds*n_regions))
    for i,simconfig in enumerate(simconfigs):
        data = []
        for j,seed in enumerate(seeds_completed[i]):
            data.append(retrieve_one_sim(simconfig, seed, root=simu_path))
        data = np.array(data)
        means[i] = reduce(data, 's t r -> (s r)', np.mean)
        stds[i] = reduce(data, 's t r -> (s r)', np.std)
    # Compare averages & std
    baseline_mean_mean = np.mean(means[0])
    mean_differences[e_idx,i_idx] = (np.mean(means[1]) - baseline_mean_mean) / baseline_mean_mean
    mean_pvalues = ttest_ind(means[0], means[1], equal_var=False).pvalue
    baseline_mean_std = np.mean(stds[0])
    std_differences[e_idx,i_idx] = (np.mean(stds[1]) - baseline_mean_std) / baseline_mean_std
    std_pvalues = ttest_ind(stds[0], stds[1], equal_var=False).pvalue

    simu_data = None
            
    # Plot for the first seeds
    print('Plot mean power')
    plot_mean_power(
        simconfigs, seeds=[0,np.load(seeds_no_exp_path)[e_idx,i_idx,0]], ignore_exists=True,
        root=simu_path, figsize=(10,10), 
        labels=x, color1s=colors, color2s=colors,
        save=2, save_path=f'{imgs_path}/PSDs', regions=None,
        file_name=f'freq_{E_L_e_end}_{E_L_i_end}_{[0,np.load(seeds_no_exp_path)[e_idx,i_idx,0]]}_seed.png')

    plot_mean_power_seeds(
        simconfigs, seeds_sets=[range(n_minimal_seeds),np.load(seeds_no_exp_path)[e_idx,i_idx]], ignore_exists=True,
        root=simu_path, figsize=(10,10), 
        labels=x, color1s=colors, color2s=colors,
        save=2, save_path=f'{imgs_path}/PSDs', regions=None,
        file_name=f'freq_{E_L_e_end}_{E_L_i_end}_{n_minimal_seeds}_seeds.png')
    
    # Diff & pvalue (computed for first seed that didn't explode for each simconfig)
    print('Diff & pvalue')
    print('Computing for params', E_L_e_end, E_L_i_end)

    seeds_sets=[range(n_minimal_seeds),np.load(seeds_no_exp_path)[e_idx,i_idx]]
    channel_powers = np.zeros((len(simconfigs),len(BAND_NAMES), n_minimal_seeds, 68))
    for s_id,(simconfig,seeds_set) in enumerate(zip(simconfigs, seeds_sets)):
        for s,seed in enumerate(seeds_set):
            _,name = simconfig.get_sim_name(seed)
            frq = np.load(os.path.join(simu_path,'freq_'+name+'.npy'))
            power_simconfig = np.load(os.path.join(simu_path,'power_'+name+'.npy'))
            for i,(low_sep,high_sep,freq_name) in enumerate(zip(SEPARATIONS,SEPARATIONS[1:],BAND_NAMES)):
                low_sep_idx = np.argmin(np.abs(low_sep - frq))
                high_sep_idx = np.argmin(np.abs(high_sep - frq))
                channel_powers[s_id,i,s] = reduce(power_simconfig[low_sep_idx:high_sep_idx], 'f r -> r', np.mean)
    channel_powers = rearrange(channel_powers, 's1 b s2 r -> s1 b (s2 r)')
    for i,(low_sep,high_sep,freq_name) in enumerate(zip(SEPARATIONS,SEPARATIONS[1:],BAND_NAMES)):
        low_sep_idx = np.argmin(np.abs(low_sep - frq))
        high_sep_idx = np.argmin(np.abs(high_sep - frq))
        data1 = channel_powers[0,i]
        data2 = channel_powers[1,i]
        frequency_differences[e_idx,i_idx,i] = (np.mean(data2) - np.mean(data1)) / np.mean(data1)
        pvalue = ttest_ind(data1, data2, equal_var=False).pvalue
        frequency_pvalues[e_idx,i_idx,i] = pvalue
    
    # Entropy computation
    print('Computing entropies')
    entropies = np.zeros((len(entropy_names),2,n_minimal_seeds))
    for i,entropy_name in enumerate(entropy_names):
        print(entropy_name)
        for j,simconfig in enumerate(simconfigs):
            data = None if simu_data is None else simu_data[j]
            entropies[i,j] = parallelized_entropy(
                simconfig, seeds=seeds_completed[j], data=data, entropy_name=entropy_name, n_tasks_concurrent=3, 
                save_path=simu_path, simu_path=simu_path)
            
        img_name = f'{entropy_name}_E_L_e_end_{E_L_e_end}_E_L_i_end_{E_L_i_end}_{n_minimal_seeds}_seeds.png'
        
        entropy_means = reduce(entropies[i], 'p s -> p', np.mean)
        entropy_deltas[i,e_idx,i_idx] = (entropy_means[1] - entropy_means[0])/entropy_means[0]
        entropy_pvalues[i,e_idx,i_idx] = ttest_ind(entropies[i][0], entropies[i][1], equal_var=False).pvalue

        print(f'{entropy_name}: {entropy_means} with pvalue {entropy_pvalues[i,e_idx,i_idx]}')

        entropy_img_name = f'{entropy_name}_E_L_e_end_{E_L_e_end}_E_L_i_end_{E_L_i_end}_pval_{entropy_pvalues[i,e_idx,i_idx]}_{n_minimal_seeds}_seeds.png'
        plot_box_points_violin(
            simconfigs=simconfigs,
            file_prefix=entropy_name,
            n_seeds=n_minimal_seeds,
            figsize=(10,8),
            save=2,
            path=simu_path,
            x=x,
            boxplot_params=boxplot_params,
            save_path=imgs_path,
            file_name=entropy_img_name,
            box_colors=colors
        )

    # PCI
    pci_simconfigs = simconfigs_setup['pci_nopsi'], simconfigs_setup['pci_psi']

    noexp_pci_seeds = np.zeros((len(pci_simconfigs), n_minimal_seeds_pci), dtype=int)
    for i,simconfig in enumerate(pci_simconfigs):
        noexp_pci_seeds[i] = run_n_complete_parallelized(
            simconfig, max_seed=len(pci_seeds), save_path_root=simu_path, n_minimal_seeds=n_minimal_seeds_pci, n_tasks_concurrent=5,
            param_change_with_seed=param_change_with_seed)
    
    pcis = np.zeros((2,n_minimal_seeds_pci))
    for j,simconfig in enumerate(pci_simconfigs):
        pcis[j] = parallelized_PCI(
            simconfig, noexp_pci_seeds[j].astype(int), 5, t_analysis=t_analysis, n_trials=n_trials, 
            simu_path=simu_path, save=1, save_path=simu_path)

    pci_means = reduce(pcis, 'p s -> p', np.mean)
    pci_deltas[e_idx,i_idx] = (pci_means[1] - pci_means[0])/pci_means[0]
    pci_test = ttest_ind(pcis[0], pcis[1], equal_var=False)
    pci_pvalues[e_idx,i_idx] = pci_test.pvalue
    
    
    set_params = {
        'title': f'PCI Stimregion {stimregion}, Stimval {stimval}',
        'ylabel': 'PCI',
        'xlabel': 'Group'
    }

    plot_box_points_violin(
        simconfigs=pci_simconfigs,
        file_prefix='PCI',
        data=pcis,
        n_seeds=n_minimal_seeds_pci,
        figsize=(10,10),
        save=2,
        path=simu_path,
        x=x,
        boxplot_params=boxplot_params,
        save_path=imgs_path,
        file_name=f'PCI_{cut_transient_pci}_{run_sim_pci}_stimregion_{stimregion}_stimval_{stimval}_E_L_e_end_{E_L_e_end}_E_L_i_end_{E_L_i_end}_seeds_{n_minimal_seeds_pci}_pval_{pci_test.pvalue}.png',
        box_colors=colors,
        set_params=set_params
    )

    
    np.savetxt(os.path.join(imgs_path, 'config_done.npy'), np.zeros(1))

In [ ]:
np.savetxt(f'{simu_path}/ai_explosions.txt', ai_explosions)
np.savetxt(f'{simu_path}/pci_explosions.txt', pci_explosions)
np.savetxt(f'{simu_path}/mean_deltas.txt', mean_differences)
np.savetxt(f'{simu_path}/mean_pvalues.txt', mean_pvalues)
np.savetxt(f'{simu_path}/std_deltas.txt', std_differences)
np.savetxt(f'{simu_path}/std_pvalues.txt', std_pvalues)
for i,entropy_name in enumerate(entropy_names):
    np.savetxt(f'{simu_path}/{entropy_name}_deltas.txt', entropy_deltas[i])
    np.savetxt(f'{simu_path}/{entropy_name}_pvalues.txt', entropy_pvalues[i])
np.savetxt(f'{simu_path}/pci_deltas.txt', pci_deltas)
np.savetxt(f'{simu_path}/pci_pvalues.txt', pci_pvalues)
for i, band_name in enumerate(BAND_NAMES):
    np.savetxt(f'{simu_path}/{band_name}_frequency_deltas.txt', frequency_differences[:,:,i])
    np.savetxt(f'{simu_path}/{band_name}_frequency_pvalues.txt', frequency_pvalues[:,:,i])

In [ ]:
im = plot_array_with_annotations(entropy_deltas[0], entropy_pvalues[0], E_L_e_ends, E_L_i_ends,
                                 axis_fontsize=8, figsize=(12,7), label='(reduced K+ leak conductance LZC - baseline LZC) / baseline LZC',
                                 save=1, save_path=imgs_path_root, save_name='LZc_matrix.pdf')
im = plot_array_with_annotations(pci_deltas, pci_pvalues, E_L_e_ends, E_L_i_ends,
                                 axis_fontsize=8, figsize=(12,7), label='(reduced K+ leak conductance PCI - baseline PCI) / baseline PCI',
                                 save=1, save_path=imgs_path_root, save_name='PCI_matrix.pdf')
for i,band_name in enumerate(BAND_NAMES):
    im = plot_array_with_annotations(frequency_differences[:,:,i], frequency_pvalues[:,:,i], E_L_e_ends, E_L_i_ends,
                                     axis_fontsize=8, figsize=(12,7), 
                                     label=f'(reduced K+ leak conductance {band_name} power - baseline {band_name} power) / baseline {band_name} power',
                                     save=1, save_path=imgs_path_root, save_name=f'{band_name}_band_matrix.pdf')    